In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import statsmodels.api as sm
import plotly.graph_objects as go

%matplotlib inline

In [2]:
data = pd.read_csv(r'C:/Users/Dell/Desktop/LOS/los_new.csv')

In [3]:
def baby_type1(x):
    #x = x.fillna(0)
    if x == 'Twins' or x=='Triplets':
        return 'Multiple'
    else:
        return 'Single'

In [4]:
data['baby_type'] = data['baby_type'].apply(baby_type1)

In [5]:
def get_dum(x):
    for elem in qs[x].unique():
        qs[str(elem)] = qs[x] == elem

In [6]:
qs = data[['uhid','Gender','MODE_OF_DELIVERY','inout_patient_status','Gestation','LOS','NEOFAX_ERRORS','MED_ADMIN_ERRORS','ESPGHAN','NUTR_ADMIN','RDS',
       'JAUNDICE', 'SEPSIS','baby_type','ANTENATA_STEROIDS']]

In [7]:
def conv(x):
    if x == True:
        return 1
    else:
        return 0

In [8]:
get_dum('inout_patient_status')
qs.drop('inout_patient_status',axis=1,inplace=True)
get_dum('NEOFAX_ERRORS')
qs = qs.rename(columns={"No_err": "NF No_err", "ERROR": "NF ERROR"})
qs.drop('NEOFAX_ERRORS',axis=1,inplace=True)
get_dum('MED_ADMIN_ERRORS')
qs = qs.rename(columns={"No_err": "MED ADMIN No_err", "ERROR": "MED ADMIN ERROR", "Not eligible" : "MED ADMIN Not eligible"})
qs.drop('MED_ADMIN_ERRORS',axis=1,inplace=True)
get_dum('ESPGHAN')
qs = qs.rename(columns={"No_err": "ESPGHAN No_err", "ERROR": "ESPGHAN ERROR","Not eligible":"ESPGHAN NE"})
get_dum('NUTR_ADMIN')
qs = qs.rename(columns={"No_err": "NUTRI ADMIN No_err", "ERROR": "NUTRI ADMIN ERROR","Not eligible":"NUTRI ADMIN NE"})
get_dum('Gender')
get_dum('RDS')
qs = qs.rename(columns={"False": "RDS False", "True": "RDS True"})
get_dum('JAUNDICE')
qs = qs.rename(columns={"False": "Jaundice False", "True": "Jaundice True"})
get_dum('SEPSIS')
qs = qs.rename(columns={"False": "Sepsis False", "True": "Sepsis True"})
get_dum('MODE_OF_DELIVERY')
get_dum('baby_type')
get_dum('ANTENATA_STEROIDS')
qs = qs.rename(columns={"false": "Antenatal ST False", "true": "Antenatal ST True", "unknown":"Antenatal ST Unknown"})
qs.drop('MODE_OF_DELIVERY',axis=1,inplace=True)
qs.drop('baby_type',axis=1,inplace=True)
#qs.drop('nan',axis=1,inplace=True)
qs.drop('RDS',axis=1,inplace=True)
qs.drop('JAUNDICE',axis=1,inplace=True)
qs.drop('SEPSIS',axis=1,inplace=True)
qs.drop('NUTR_ADMIN',axis=1,inplace=True)
qs.drop('ESPGHAN',axis=1,inplace=True)
qs.drop('Gender',axis=1,inplace=True)
qs.drop('ANTENATA_STEROIDS',axis=1,inplace=True)

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [15]:
qs.columns

Index(['uhid', 'Gestation', 'LOS', 'In Born', 'Out Born',
       'MED ADMIN Not eligible', 'NF No_err', 'NF ERROR', 'MED ADMIN ERROR',
       'MED ADMIN No_err', 'ESPGHAN ERROR', 'ESPGHAN No_err', 'ESPGHAN NE',
       'NUTRI ADMIN NE', 'NUTRI ADMIN No_err', 'NUTRI ADMIN ERROR', 'Male',
       'Female', 'RDS False', 'RDS True', 'Jaundice True', 'Jaundice False',
       'Sepsis False', 'Sepsis True', 'NVD', 'LSCS', 'Single', 'Multiple',
       'Antenatal ST True', 'Antenatal ST False', 'Antenatal ST Unknown'],
      dtype='object')

In [9]:
l = ['In Born', 'Out Born', 'NF No_err', 'NF ERROR',
       'MED ADMIN Not eligible', 'MED ADMIN No_err', 'MED ADMIN ERROR',
       'ESPGHAN ERROR', 'ESPGHAN No_err','ESPGHAN NE', 'NUTRI ADMIN ERROR',
       'NUTRI ADMIN No_err','NUTRI ADMIN NE' ,'Male', 'Female', 'RDS False', 'RDS True',
       'Jaundice False', 'Jaundice True', 'Sepsis False', 'Sepsis True',
       'LSCS', 'NVD', 'Single', 'Multiple', 'Antenatal ST False',
       'Antenatal ST True', 'Antenatal ST Unknown']

In [10]:
for i in l:
        qs[i] = qs[i].apply(conv)

In [19]:
def model(data):
    y = data['LOS']
    X = data.drop('LOS',axis=1)
    X.drop('uhid',axis=1,inplace=True)
    NegBi_model = sm.GLM(y,X, family=sm.families.NegativeBinomial())
    NegBi_results = NegBi_model.fit()
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]

    return (NegBi_results.params, dt)


In [30]:
data_34 = qs[qs['Gestation']<34]
data_36 = qs[(qs['Gestation']>=34) & (qs['Gestation']<=37)]
data_37 = qs[qs['Gestation']>37]

In [13]:
z, m = model(data_37)

In [40]:
m

,coef,std err,z,P>|z|,[0.025,0.975]
Gestation,-0.0050,0.043,-0.116,0.908,-0.089,0.079
In Born,0.0657,0.174,0.378,0.706,-0.275,0.407
Out Born,0.2362,0.182,1.295,0.195,-0.121,0.594
MED ADMIN Not eligible,0.2153,0.131,1.650,0.099,-0.041,0.471
NF No_err,0.2727,0.146,1.874,0.061,-0.013,0.558
NF ERROR,0.5072,0.151,3.366,0.001,0.212,0.803
MED ADMIN ERROR,0.0589,0.157,0.376,0.707,-0.249,0.367
MED ADMIN No_err,0.0277,0.183,0.152,0.879,-0.330,0.385
ESPGHAN ERROR,0.1496,0.131,1.142,0.254,-0.107,0.407
ESPGHAN No_err,0.1899,0.144,1.316,0.188,-0.093,0.473


In [14]:
def refgrid(x):

    gest = [x[0]]
    inout = [x[2],x[1]]
    NF = [x[5],x[4]]
    med_admin = [x[7],x[6]]
    espghan = [x[8],x[9]]
    nutri = [x[12],x[13]]
    gender = [x[14],x[15]]
    rds = [x[17],x[16]]
    jaundice = [x[18],x[19]]
    sepsis = [x[21],x[20]]
    typelsnv = [x[23],x[22]]
    type123 = [x[24],x[25]]
    anst = [x[26],x[27]]
    los_combo = list(itertools.product(inout,NF,med_admin,espghan,gender,nutri,rds,jaundice,sepsis,typelsnv,type123,anst,gest))
    los_df = pd.DataFrame(los_combo)
    los_df = los_df.rename(columns={0: "inout", 1: "NF", 2 : "Med_admin", 3: "espghan", 4:"nutri", 5:"gender", 6:"rds", 7:"jaundice", 8:"sepsis",9:"lscs_nvd",10:"birth_type",11:"ant_std",12:"Gestation"})
    return los_df

In [15]:
def choice34_37(x):
    z, m = model(data_36)
    
    d = []
    e = []
    a = refgrid(z)
    q ={"inout":1, "NF":4, "Med_admin":6, "espghan":8, "nutri":12, "gender":14, "rds":16, "jaundice":18, "sepsis":20, "lscs_nvd":22, "birth_type":24,"ant_std":27}
    p = q[x]
    
    los = a
    los_new = a.drop(str(x),axis=1)
    los_1 = los_new.drop('Gestation',axis=1)
    los_1['sum'] = los_new.sum(axis=1)
    
    
    
    
    if m['P>|z|'][p] < .05 and m['P>|z|'][p+1] < .05:
           
        #los1 = qs[qs['Gestation']<34]
        #median1 = np.median(los1['LOS'])
        
        for j in range(len(los_new)):
            for i in np.arange(34.0,37.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
        s = pd.DataFrame(d)
        t = pd.DataFrame(e)
        
        return(round(np.exp(s.mean())[0]-data_36.median()[1]),round(np.exp(t.mean())[0]-data_36.median()[1]))
    elif m['P>|z|'][p] < .05 and m['P>|z|'][p+1] > .05:
        for j in range(len(los_new)):
            for i in np.arange(34.0,37.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(round(np.exp(s.mean())[0]-data_36.median()[1]),' ')
    elif m['P>|z|'][p] > .05 and m['P>|z|'][p+1] < .05:
        for j in range(len(los_new)):
            for i in np.arange(34.0,37.0,0.1):
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
                d.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(' ',round(np.exp(t.mean())[0]))
                
        
        
        
    else:
        return (' ', ' ')

In [16]:
def choice25_34(x):
    z, m = model(data_34)
    
    d = []
    e = []
    a = refgrid(z)
    q ={"inout":1, "NF":4, "Med_admin":6, "espghan":8, "nutri":12, "gender":14, "rds":16, "jaundice":18, "sepsis":20, "lscs_nvd":22, "birth_type":24,"ant_std":27}
    p = q[x]
    
    los = a
    los_new = a.drop(str(x),axis=1)
    los_1 = los_new.drop('Gestation',axis=1)
    los_1['sum'] = los_new.sum(axis=1)
    
    
    
    
    if m['P>|z|'][p] < .05 and m['P>|z|'][p+1] < .05:
           
        #los1 = qs[qs['Gestation']<34]
        #median1 = np.median(los1['LOS'])
        
        for j in range(len(los_new)):
            for i in np.arange(25.0,34.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
        s = pd.DataFrame(d)
        t = pd.DataFrame(e)
        
        return(round(np.exp(s.mean())[0]-data_34.median()[1]),round(np.exp(t.mean())[0]-data_34.median()[1]))
    elif m['P>|z|'][p] < .05 and m['P>|z|'][p+1] > .05:
        for j in range(len(los_new)):
            for i in np.arange(25.0,34.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(round(np.exp(s.mean())[0]-data_34.median()[1]),' ')
    elif m['P>|z|'][p] > .05 and m['P>|z|'][p+1] < .05:
        for j in range(len(los_new)):
            for i in np.arange(25.0,34.0,0.1):
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
                d.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(' ',round(np.exp(t.mean())[0]))
                
        
        
        
    else:
        return (' ', ' ')

In [17]:
def choice37_44(x):
    z, m = model(data_37)
    
    d = []
    e = []
    a = refgrid(z)
    q ={"inout":1, "NF":4, "Med_admin":6, "espghan":8, "nutri":12, "gender":14, "rds":16, "jaundice":18, "sepsis":20, "lscs_nvd":22, "birth_type":24,"ant_std":27}
    p = q[x]
    
    los = a
    los_new = a.drop(str(x),axis=1)
    los_1 = los_new.drop('Gestation',axis=1)
    los_1['sum'] = los_new.sum(axis=1)
    
    
    
    
    if m['P>|z|'][p] < .05 and m['P>|z|'][p+1] < .05:
           
        #los1 = qs[qs['Gestation']<34]
        #median1 = np.median(los1['LOS'])
        
        for j in range(len(los_new)):
            for i in np.arange(37.0,44.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
        s = pd.DataFrame(d)
        t = pd.DataFrame(e)
        
        return(round(np.exp(s.mean())[0]-data_37.median()[1]),round(np.exp(t.mean())[0]-data_37.median()[1]))
    elif m['P>|z|'][p] < .05 and m['P>|z|'][p+1] > .05:
        for j in range(len(los_new)):
            for i in np.arange(37.0,44.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(round(np.exp(s.mean())[0]-data_37.median()[1]),' ')
    elif m['P>|z|'][p] > .05 and m['P>|z|'][p+1] < .05:
        for j in range(len(los_new)):
            for i in np.arange(37.0,44.0,0.1):
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
                d.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(' ',round(np.exp(t.mean())[0]))
                
        
        
        
    else:
        return (' ', ' ')

In [82]:
choice25_34('NF')

(' ', 17.0)

In [74]:
choice34_37('inout')

(2.0, 3.0)

In [20]:
choice37_44('inout')

(' ', ' ')

In [21]:
def median_IQR(x):
    q1 = str(pd.DataFrame(x).quantile(.25)[0])
    q2 = str(pd.DataFrame(x).quantile(.75)[0])
    q3 = q1 +" to " + q2
    return q3

In [31]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>FIELDS</b>','<b>Less than 34</b>','<b>34 to 37</b>','<b>Greater than 37</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Count','Median','IQR','Male','Female', 'Out born','In Born', 'LSCS','NVD','Single' ,'Multiple', 'Antenatal Steroids False','Antenatal Steroids True','NeoFax No Error','NeoFax Error','Med Admin Error','Med Admin No Error','ESPHGAN Error','ESPGHAN No Error','Nutri Admin No Error','Nutri Admin Error','RDS True','RDS False','Jaundice False','Jaundice True','Sepsis True','Sepsis False'],
      [data_34.count()[0],data_34.median()[1],median_IQR(data_34['LOS']),choice25_34('gender')[0],choice25_34('gender')[1], choice25_34('inout')[0],choice25_34('inout')[1], choice25_34('lscs_nvd')[0],choice25_34('lscs_nvd')[1], choice25_34('birth_type')[0],choice25_34('birth_type')[1],choice25_34('ant_std')[1],choice25_34('ant_std')[0], choice25_34('NF')[0],choice25_34('NF')[1],choice25_34('Med_admin')[1],choice25_34('Med_admin')[0],choice25_34('espghan')[0],choice25_34('espghan')[1],choice25_34('nutri')[1],choice25_34('nutri')[0],choice25_34('rds')[0],choice25_34('rds')[1],choice25_34('jaundice')[0],choice25_34('jaundice')[1],choice25_34('sepsis')[0],choice25_34('sepsis')[1]],
      [data_36.count()[0],data_36.median()[1],median_IQR(data_36['LOS']),choice34_37('gender')[0],choice34_37('gender')[1], choice34_37('inout')[0],choice25_34('inout')[1], choice34_37('lscs_nvd')[0],choice34_37('lscs_nvd')[1], choice34_37('birth_type')[0],choice34_37('birth_type')[1],choice34_37('ant_std')[1],choice34_37('ant_std')[0], choice34_37('NF')[0],choice34_37('NF')[1],choice34_37('Med_admin')[1],choice34_37('Med_admin')[0],choice34_37('espghan')[0],choice34_37('espghan')[1],choice34_37('nutri')[1],choice34_37('nutri')[0],choice34_37('rds')[0],choice34_37('rds')[1],choice34_37('jaundice')[0],choice34_37('jaundice')[1],choice34_37('sepsis')[0],choice34_37('sepsis')[1]],
      [data_37.count()[0],data_37.median()[1],median_IQR(data_37['LOS']),choice37_44('gender')[0],choice37_44('gender')[1], choice37_44('inout')[0],choice25_34('inout')[1], choice37_44('lscs_nvd')[0],choice37_44('lscs_nvd')[1], choice37_44('birth_type')[0],choice37_44('birth_type')[1],choice37_44('ant_std')[1],choice37_44('ant_std')[0], choice37_44('NF')[0],choice37_44('NF')[1],choice37_44('Med_admin')[1],choice37_44('Med_admin')[0],choice37_44('espghan')[0],choice37_44('espghan')[1],choice37_44('nutri')[1],choice37_44('nutri')[0],choice37_44('rds')[0],choice37_44('rds')[1],choice37_44('jaundice')[0],choice37_44('jaundice')[1],choice37_44('sepsis')[0],choice37_44('sepsis')[1]],

    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.show()

KeyboardInterrupt: 

In [33]:
Dict1 = {'Less than 34' : {'Count': str(data_34.count()[0]), 'Median': str(data_34.median()[1]),'IQR': median_IQR(data_34['LOS']),'Male':choice25_34('gender')[0],'Female' : choice25_34('gender')[1],'Out Born' : choice25_34('inout')[0],'In Born' : choice25_34('inout')[1], 'LSCS' : choice25_34('lscs_nvd')[0],'NVD' : choice25_34('lscs_nvd')[1],'Single' : choice25_34('birth_type')[0], 'Multiple' : choice25_34('birth_type')[1],'Antenatal Steroids False' : choice25_34('ant_std')[1],'Antenatal Steroids True' : choice25_34('ant_std')[0],'NeoFax No Error' : choice25_34('NF')[0],'NeoFax Error' : choice25_34('NF')[1],'Med Admin Error' : choice25_34('Med_admin')[1],'Med Admin No Error' : choice25_34('Med_admin')[0],'ESPGHAN Error' : choice25_34('espghan')[0],'ESPGHAN No Error' : choice25_34('espghan')[1],'Nutri Admin No Error' : choice25_34('nutri')[1],'Nutri Admin Error' : choice25_34('nutri')[0],'RDS True': choice25_34('rds')[0],'RDS False':choice25_34('rds')[1],'Jaundice False':choice25_34('jaundice')[0],'Jaundice True':choice25_34('jaundice')[1],'Sepsis True':choice25_34('sepsis')[0],'Sepsis False':choice25_34('sepsis')[1]},
        '34 to 37' : {'Count': str(data_36.count()[0]), 'Median': str(data_36.median()[1]),'IQR': median_IQR(data_36['LOS']),'Male':choice34_37('gender')[0],'Female' : choice34_37('gender')[1],'Out Born' : choice34_37('inout')[0],'In Born' : choice34_37('inout')[1], 'LSCS' : choice34_37('lscs_nvd')[0],'NVD' : choice34_37('lscs_nvd')[1],'Single' : choice34_37('birth_type')[0], 'Multiple' : choice34_37('birth_type')[1],'Antenatal Steroids False' : choice34_37('ant_std')[1],'Antenatal Steroids True' : choice34_37('ant_std')[0],'NeoFax No Error' : choice34_37('NF')[0],'NeoFax Error' : choice34_37('NF')[1],'Med Admin Error' : choice34_37('Med_admin')[1],'Med Admin No Error' : choice34_37('Med_admin')[0],'ESPGHAN Error' : choice34_37('espghan')[0],'ESPGHAN No Error' : choice34_37('espghan')[1],'Nutri Admin No Error' : choice34_37('nutri')[1],'Nutri Admin Error' : choice34_37('nutri')[0],'RDS True': choice34_37('rds')[0],'RDS False':choice34_37('rds')[1],'Jaundice False':choice34_37('jaundice')[0],'Jaundice True':choice34_37('jaundice')[1],'Sepsis True':choice34_37('sepsis')[0],'Sepsis False':choice34_37('sepsis')[1]},
        'Greater than 37' : {'Count': str(data_37.count()[0]), 'Median': str(data_37.median()[1]),'IQR': median_IQR(data_37['LOS']),'Male':choice37_44('gender')[0],'Female' : choice37_44('gender')[1],'Out Born' : choice37_44('inout')[0],'In Born' : choice37_44('inout')[1], 'LSCS' : choice37_44('lscs_nvd')[0],'NVD' : choice37_44('lscs_nvd')[1],'Single' : choice37_44('birth_type')[0], 'Multiple' : choice37_44('birth_type')[1],'Antenatal Steroids False' : choice37_44('ant_std')[1],'Antenatal Steroids True' : choice37_44('ant_std')[0],'NeoFax No Error' : choice37_44('NF')[0],'NeoFax Error' : choice37_44('NF')[1],'Med Admin Error' : choice37_44('Med_admin')[1],'Med Admin No Error' : choice37_44('Med_admin')[0],'ESPGHAN Error' : choice37_44('espghan')[0],'ESPGHAN No Error' : choice37_44('espghan')[1],'Nutri Admin No Error' : choice37_44('nutri')[1],'Nutri Admin Error' : choice37_44('nutri')[0],'RDS True': choice37_44('rds')[0],'RDS False':choice37_44('rds')[1],'Jaundice False':choice37_44('jaundice')[0],'Jaundice True':choice37_44('jaundice')[1],'Sepsis True':choice37_44('sepsis')[0],'Sepsis False':choice37_44('sepsis')[1]}}
        
        


In [34]:
Dict1

{'Less than 34': {'Count': '319',
  'Median': '12.0',
  'IQR': '5.0 to 25.0',
  'Male': -10.0,
  'Female': -10.0,
  'Out Born': -9.0,
  'In Born': -10.0,
  'LSCS': -10.0,
  'NVD': -10.0,
  'Single': -9.0,
  'Multiple': -10.0,
  'Antenatal Steroids False': -10.0,
  'Antenatal Steroids True': -9.0,
  'NeoFax No Error': -10.0,
  'NeoFax Error': ' ',
  'Med Admin Error': -9.0,
  'Med Admin No Error': -10.0,
  'ESPGHAN Error': -10.0,
  'ESPGHAN No Error': -9.0,
  'Nutri Admin No Error': -10.0,
  'Nutri Admin Error': -10.0,
  'RDS True': -10.0,
  'RDS False': -10.0,
  'Jaundice False': -9.0,
  'Jaundice True': -10.0,
  'Sepsis True': -10.0,
  'Sepsis False': -9.0},
 '34 to 37': {'Count': '310',
  'Median': '2.0',
  'IQR': '1.0 to 4.0',
  'Male': ' ',
  'Female': ' ',
  'Out Born': ' ',
  'In Born': ' ',
  'LSCS': ' ',
  'NVD': ' ',
  'Single': ' ',
  'Multiple': ' ',
  'Antenatal Steroids False': ' ',
  'Antenatal Steroids True': ' ',
  'NeoFax No Error': 2.0,
  'NeoFax Error': 2.0,
  'Med A

In [86]:
model(data_34)

(Gestation                -0.308973
 In Born                   1.260941
 Out Born                  1.238806
 MED ADMIN Not eligible    0.977104
 NF No_err                 0.241765
 NF ERROR                  0.311539
 MED ADMIN ERROR           0.767714
 MED ADMIN No_err          0.754928
 ESPGHAN ERROR             1.010534
 ESPGHAN No_err            0.991194
 ESPGHAN NE                0.498019
 NUTRI ADMIN NE            0.949899
 NUTRI ADMIN No_err        0.781334
 NUTRI ADMIN ERROR         0.768514
 Male                      1.243516
 Female                    1.256231
 RDS False                 1.233769
 RDS True                  1.265978
 Jaundice True             1.335973
 Jaundice False            1.163774
 Sepsis False              1.205746
 Sepsis True               1.294001
 NVD                       1.247856
 LSCS                      1.251891
 Single                    1.254989
 Multiple                  1.244758
 Antenatal ST True         0.849081
 Antenatal ST False        0

In [96]:
def model1(data):
    y = data['LOS']
    X = data.drop('LOS',axis=1)
    X.drop('uhid',axis=1,inplace=True)
    NegBi_model = sm.GLM(y,X, family=sm.families.NegativeBinomial())
    NegBi_results = NegBi_model.fit()
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]
    sst_val = sum(map(lambda x: np.power(X,2),y-np.mean(y))) 

    sse_val = sum(map(lambda x: np.power(X,2),NegBi_results.resid_response)) 

    r2 = 1.0 - sse_val/sst_val

    return (NegBi_results.summary())


In [97]:
model1(data_34)

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    LOS   No. Observations:                  514
Model:                            GLM   Df Residuals:                      495
Model Family:        NegativeBinomial   Df Model:                           18
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1870.7
Date:                Thu, 22 Aug 2019   Deviance:                       212.09
Time:                        16:45:44   Pearson chi2:                     231.
No. Iterations:                    10   Covariance Type:             nonrobust
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Gestation                 -0.3090      0.029    -10.742      0.000      -0.365      -0.253
In Born                    1.2609      0.103     12.271      0.000       1.060       1.462
Out Born                   1.2388      0.117     10.605      0.000       1.010       1.468
MED ADMIN Not eligible     0.9771      0.102      9.600      0.000       0.778       1.177
NF No_err                  0.2418      0.157      1.544      0.122      -0.065       0.549
NF ERROR                   0.3115      0.157      1.982      0.047       0.004       0.620
MED ADMIN ERROR            0.7677      0.124      6.171      0.000       0.524       1.012
MED ADMIN No_err           0.7549      0.126      5.991      0.000       0.508       1.002
ESPGHAN ERROR              1.0105      0.106      9.575      0.000       0.804       1.217
ESPGHAN No_err             0.9912      0.093     10.644      0.000       0.809       1.174
ESPGHAN NE                 0.4980      0.116      4.302      0.000       0.271       0.725
NUTRI ADMIN NE             0.9499      0.105      9.059      0.000       0.744       1.155
NUTRI ADMIN No_err         0.7813      0.092      8.530      0.000       0.602       0.961
NUTRI ADMIN ERROR          0.7685      0.101      7.618      0.000       0.571       0.966
Male                       1.2435      0.107     11.602      0.000       1.033       1.454
Female                     1.2562      0.109     11.531      0.000       1.043       1.470
RDS False                  1.2338      0.119     10.374      0.000       1.001       1.467
RDS True                   1.2660      0.103     12.282      0.000       1.064       1.468
Jaundice True              1.3360      0.104     12.887      0.000       1.133       1.539
Jaundice False             1.1638      0.112     10.410      0.000       0.945       1.383
Sepsis False               1.2057      0.119     10.101      0.000       0.972       1.440
Sepsis True                1.2940      0.114     11.324      0.000       1.070       1.518
NVD                        1.2479      0.112     11.107      0.000       1.028       1.468
LSCS                       1.2519      0.117     10.745      0.000       1.024       1.480
Single                     1.2550      0.118     10.665      0.000       1.024       1.486
Multiple                   1.2448      0.102     12.263      0.000       1.046       1.444
Antenatal ST True          0.8491      0.110      7.727      0.000       0.634       1.064
Antenatal ST False         0.6095      0.128      4.760      0.000       0.359       0.860
Antenatal ST Unknown       1.0412      0.102     10.247      0.000       0.842       1.240
==========================================================================================
"""

In [25]:
def apollo(x):
    if x.split('.')[0] == 'RSHI':
        return 1
    else:
        return 0

In [26]:
qs['apollo'] = qs['uhid'].apply(apollo)

In [99]:
-2*(-1870.7) + 2*29

3799.4

In [28]:
qs = qs[qs['apollo']==1]

In [29]:
qs

,uhid,Gestation,LOS,In Born,Out Born,MED ADMIN Not eligible,NF No_err,NF ERROR,MED ADMIN ERROR,MED ADMIN No_err,...,Sepsis False,Sepsis True,NVD,LSCS,Single,Multiple,Antenatal ST True,Antenatal ST False,Antenatal ST Unknown,apollo
1,RSHI.0000019414,35.142857,4,0,1,0,0,0,1,0,...,1,0,0,1,1,0,0,1,0,1
2,RSHI.0000019145,37.571429,2,1,0,1,0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
7,RSHI.0000019298,34.142857,3,1,0,0,1,0,1,0,...,0,1,0,1,1,0,0,1,0,1
8,RSHI.0000019213,39.428571,1,1,0,1,0,0,0,0,...,1,0,0,1,1,0,0,1,0,1
10,RSHI.0000019134,36.428571,1,1,0,0,1,0,0,1,...,0,1,1,0,1,0,0,0,1,1
13,RSHI.0000019186,35.857143,1,1,0,1,0,0,0,0,...,1,0,0,1,1,0,0,0,1,1
14,RSHI.0000019147,37.000000,2,1,0,1,0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
15,RSHI.0000019145,37.571429,2,1,0,1,0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
16,RSHI.0000019134,36.428571,1,1,0,0,1,0,0,1,...,0,1,1,0,1,0,0,0,1,1
18,RSHI.0000019149,34.000000,2,0,1,1,0,0,0,0,...,1,0,0,1,0,1,0,1,0,1
